# Playing Simple Games with Neural Nets

In this notebook, we implement equilibria learning viea self play for simple games such as Battle of the Sexes and Matching Pennies.

In [1]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
import torch
from bnelearn.strategy import MatrixGameStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import TwoByTwoBimatrixGame
from bnelearn.optimizer import ES
from bnelearn.environment import Environment

In [2]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## Experiment setup
n_players = 2

## Environment settings
#training batch size
batch_size = 64
input_length = 1


# optimization params
epoch = 10
learning_rate = 1
lr_decay = False
lr_decay_every = 1000
lr_decay_factor = 0.8

sigma = 5 #ES noise parameter
n_perturbations = 8

name = 13
namestr = './pd/{}'.format(name)

### Env

In [4]:
# prisoners dilemma
pd = torch.tensor([[[-1, -1],[-3, 0]], [[ 0, -3],[-2,-2]]])
prisoners_dilemma = TwoByTwoBimatrixGame(pd)

# battle of sexes. ! won't work, game is noy symmetric!
mp = torch.tensor([[[3, 2],[0,0]], [[0,0],[2,3]]])

In [5]:
# Wrapper transforming a strategy to bidder, used by the optimizer
# this is a dummy, valuation doesn't matter
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(0,0, strategy, batch_size = batch_size, n_players=2)

In [6]:
model = MatrixGameStrategy(n_actions=2).cuda()

In [7]:
game = prisoners_dilemma

In [8]:
env = Environment(game, environment_agents=[],
                 max_env_size =1,
                 n_players=2,
                 batch_size=batch_size,
                 strategy_to_bidder_closure=strat_to_bidder)

In [9]:
optimizer = ES(model=model, environment = env, lr = learning_rate, sigma=sigma, n_perturbations=n_perturbations)

In [24]:
def log_hyperparams(writer):
    writer.add_scalar('hyperparams/batch_size', batch_size)
    writer.add_scalar('hyperparams/learning_rate', learning_rate)
    writer.add_scalar('hyperparams/sigma', sigma)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations)
    
    

Training

In [25]:
torch.cuda.empty_cache()
writer = SummaryWriter(log_dir=namestr)
log_hyperparams(writer)

for e in range(epoch+1):    
    
    # lr decay?
    if lr_decay and e % lr_decay_every == 0 and e > 0:
        learning_rate = learning_rate * lr_decay_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
        writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
        
    # always: do optimizer step
    utility = -optimizer.step()
    writer.add_scalar('eval/utility', utility, e) 
    writer.add_scalar('eval/prob_action_0', model.distribution.probs[0], e)    
    #print(list(model.named_parameters()))
    print(e)
        
torch.cuda.empty_cache()
writer.close()

0
1
2
3
4
5
6
7
8
9
10


In [10]:
pd

tensor([[[-1, -1],
         [-3,  0]],

        [[ 0, -3],
         [-2, -2]]])

In [11]:
bidder = strat_to_bidder(model, batch_size)

In [12]:
b =bidder.get_action()

In [13]:
b

tensor([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], device='cuda:0')

In [14]:
ob = next(env._generate_opponent_bids())

In [15]:
bids = torch.cat((b,ob), 1).view(batch_size, n_players, 1)

In [16]:
bids[2][1][0] = 1
bids[0][0][0] = 0

In [17]:
pay = torch.zeros(batch_size, n_players)

In [18]:
pd = pd.cuda()

In [19]:
pay = pd[bids[:,0], bids[:,1]].view(batch_size, n_players)

In [20]:
alloc, pay = game.run(bids)

In [21]:
(bidder.valuations * alloc[:,0,:]).sum(dim=1) - pay[:,0]

tensor([-3.,  0., -3., -1., -3., -1., -1.,  0., -1., -1., -3., -1., -1.,  0.,
        -1., -1., -1., -1., -1., -1., -1.,  0., -1., -1., -1., -1., -1., -3.,
        -1., -1., -1., -1., -1., -1., -1., -1.,  0., -1., -1.,  0., -1., -1.,
        -1., -1.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -3.,
        -1., -1., -1., -1., -1., -1., -1., -1.], device='cuda:0')

In [22]:
pay[:,0].dtype

torch.float32